In [9]:
%pip install streamlit pandas scikit-learn nltk zipfile

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [10]:
import zipfile
import pandas as pd

zip_file_path = '/content/Suspicious Communication on Social Platforms.csv.zip'
extracted_file_path = '/content/Suspicious Communication on Social Platforms.csv'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

df = pd.read_csv(extracted_file_path)
display(df.head())

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1


In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Add download for punkt_tab

# Handle missing values
df['comments'] = df['comments'].fillna('')

# Tokenization and remove stop words
stop_words = set(stopwords.words('english'))
df['comments'] = df['comments'].apply(lambda x: ' '.join([word for word in word_tokenize(x.lower()) if word.isalpha() and word not in stop_words]))

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features
X = tfidf_vectorizer.fit_transform(df['comments']).toarray()
y = df['tagging']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data preprocessing complete.")
print("Shape of training data (X_train):", X_train.shape)
print("Shape of testing data (X_test):", X_test.shape)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Data preprocessing complete.
Shape of training data (X_train): (16000, 5000)
Shape of testing data (X_test): (4001, 5000)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Initialize and train the model (Using Logistic Regression as an example)
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Model training complete.")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Model training complete.
Accuracy: 0.75
Precision: 0.72
Recall: 0.59


In [14]:
import joblib

# Save the trained model and TF-IDF vectorizer
joblib.dump(model, 'trained_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

print("Trained model and TF-IDF vectorizer saved.")

Trained model and TF-IDF vectorizer saved.


Now I will create the Streamlit application script. You can run this script locally on your machine after downloading it from Colab.

In [15]:
%%writefile app.py
import streamlit as st
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# Download necessary NLTK data (if not already downloaded)
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')


# Load the trained model and TF-IDF vectorizer
model = joblib.load('trained_model.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Define preprocessing function
def preprocess_text(text):
    # Handle missing values
    text = text if isinstance(text, str) else ''
    # Tokenization and remove stop words
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    preprocessed_words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(preprocessed_words)

# Streamlit app title and description
st.title("Cyberbullying Detection Framework")
st.write("Enter a comment to check if it is cyberbullying.")

# Text input from user
user_input = st.text_area("Enter comment here:")

if st.button("Analyze"):
    if user_input:
        # Preprocess the user input
        preprocessed_input = preprocess_text(user_input)

        # Vectorize the preprocessed input using the loaded TF-IDF vectorizer
        input_vectorized = tfidf_vectorizer.transform([preprocessed_input]).toarray()

        # Make prediction
        prediction = model.predict(input_vectorized)

        # Display the result
        if prediction[0] == 1:
            st.error("This comment is likely cyberbullying.")
        else:
            st.success("This comment is not likely cyberbullying.")
    else:
        st.warning("Please enter a comment to analyze.")

Writing app.py
